# jiu-jitsu-auto-scoring
*This notebook is a **WIP**!*


In [1]:
# prompt: mount drive and clone the repository in the current branch, the organization is called jiu-jitsu-auto-scoring and the repository is called model
from google.colab import drive

BRANCH = "5-try-out-vitpose" # Write the name of the branch you're working in here

!git clone -q https://github.com/jiu-jitsu-auto-scoring/model.git
%cd /content/model
!git pull
!git checkout $BRANCH

/content/model
Already up to date.
Branch '5-try-out-vitpose' set up to track remote branch '5-try-out-vitpose' from 'origin'.
Switched to a new branch '5-try-out-vitpose'


#0. Imports, Requirements, etc.
Upload the required checkpoint into that folder

In [3]:
# Import drive for custom images
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!git clone https://github.com/huggingface/transformers
!cd transformers

Cloning into 'transformers'...
remote: Enumerating objects: 256362, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 256362 (delta 348), reused 247 (delta 230), pack-reused 255903 (from 4)
Receiving objects: 100% (256362/256362), 269.54 MiB | 19.78 MiB/s, done.
Resolving deltas: 100% (188826/188826), done.
Updating files: 100% (5018/5018), done.


In [5]:
!pip install ./transformers

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10742239 sha256=24cf0c49066855af02978542297e00a88b8a5cad513c12aaa09ff6c62595e80e
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_uwveb4/wheels/26/9e/af/853f5e0ae59879031d71f00ecc3e45610340ee4dd7b748a93b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2


Install gcsfs 2024.9.0 to work with datasets installation in requirements. Beware 2024.9.0 might cause issues down the road (it was yanked)

In [10]:
!pip uninstall gcsfs
!pip install gcsfs==2024.9.0

Found existing installation: gcsfs 2024.9.0
Uninstalling gcsfs-2024.9.0:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/gcsfs-2024.9.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/gcsfs/*
Proceed (Y/n)? Y
  Successfully uninstalled gcsfs-2024.9.0
  Using cached gcsfs-2024.9.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Reason for being yanked: requirements incorrect
Using cached gcsfs-2024.9.0-py2.py3-none-any.whl (34 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.9.0
    Uninstalling fsspec-2024.9.0:
      Successfully uninstalled fsspec-2024.9.0


In [11]:
!pip install -r transformers//examples/pytorch/object-detection/requirements.txt

Create a new write token on huggingface and paste the token here.

In [23]:
from huggingface_hub import login

login(token="paste token here!")

Login to Weights & Biases to track results

In [24]:
import wandb
wandb.login()

True

Start training using script. Note this automatically pushes the model to your hugging face account. If you want to rewrite make sure to delete the folder in the model directory in google colab and delete the model on your hugging face account

In [26]:
!python transformers/examples/pytorch/object-detection/run_object_detection.py \
    --model_name_or_path facebook/detr-resnet-50 \
    --dataset_name cppe-5 \
    --do_train true \
    --do_eval true \
    --output_dir detr-finetuned-cppe-5-10k-steps1 \
    --num_train_epochs 100 \
    --image_square_size 600 \
    --fp16 true \
    --learning_rate 5e-5 \
    --weight_decay 1e-4 \
    --dataloader_num_workers 4 \
    --dataloader_prefetch_factor 2 \
    --per_device_train_batch_size 8 \
    --gradient_accumulation_steps 1 \
    --remove_unused_columns false \
    --eval_do_concat_batches false \
    --ignore_mismatched_sizes true \
    --metric_for_best_model eval_map \
    --greater_is_better true \
    --load_best_model_at_end true \
    --logging_strategy epoch \
    --evaluation_strategy epoch \
    --save_strategy epoch \
    --save_total_limit 2 \
    --push_to_hub true \
    --push_to_hub_model_id detr-finetuned-cppe-5-10k-steps1 \
    --hub_strategy end \
    --seed 1337

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2025-02-12 16:33:28.275899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739378008.302029   14231 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739378008.309759   14231 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 16:33:28.335701: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimize